In [1]:
import numpy as np
import json
import glob
import pandas as pd
import csv

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Spacy
import spacy
from nltk.corpus import stopwords

# Vis
import pyLDAvis
import pyLDAvis.gensim_models
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
all_files = ["coinbase.csv", "binance.csv", "ftx.csv", "kraken.csv", "robinhood.csv", "webull.csv", "etoro.csv"]

In [4]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = pd.read_csv(file)
        # print(data)
        # data = json.load(f)
    return (data);

def write_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4);

# Combine all the csv files into one

li = []
for filename in all_files:
    df = load_data(filename)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
data = frame.loc[:, ["content", "score"]]
# data = data.loc[(data.score <= 2)] # Grab only poor reviews
content = json.loads(data.loc[:, "content"].to_json())

# Prefilter step

# Remove reviews with <= 50 chars
data = [content[key] for key in content if content[key] != None and len(content[key]) > 50];
len(data)

## Break each sentence into its own document

#

16754

In [5]:
# Stop words are basically words that are not really useful in the topic model
# i.e. words that you want to eliminate from consideration
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
# data = load_data("ushmm_dn.json")["texts"]
print(data[0][0:90]) # Prints out first text, char 0-89

Don't like to do update needs to go back to the old format..it takes to much time to look 


In [7]:
# Reducing words to their root form (lemma) using spacy or nltk to grab roots
# Specify allowed_postags for spacy

# Takes a list of texts i.e. [text1, text2...] and returns lemmatised [text1..]
# Allowed_post_tages -> Basically default arguments in python 
# This is a list initalised that is used to filter tokens 
# pos_ tag marks category of word token belongs to i.e. only append nouns,adj,verbs,and adverbs 
# e.g. conjunctions (but, and, because), and pronouns (he/she) is ignored
def lemmatization(texts, allowed_postags=["NOUN"]):

    # Spacy load creates an an nlp parser i.e. function pointer 
    spacy.load
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []

    for text in texts:
        doc = nlp(text) # Create document from the raw text/string
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags and token.pos_ not in stopwords:
               new_text.append(token.lemma_) # Appends only lemma to new_text
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out);

lemmatized_texts = lemmatization(data)
print(lemmatized_texts[0][0:90])

need format time gainer loser


In [8]:
# Simple preprocess converts documents to list of lower case tokens ignoring ones
# which are too short or too long.
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True);
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print(data_words[0][0:20])


['need', 'format', 'time', 'gainer', 'loser']


In [9]:
# BIGRAMS AND TRIGRAMS

# min_count - ignore all words/bigrams with total collected count lower than this value.
# threshold - score for forming phrases play around with this -> related to scoring function
bigrams_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=1) 

# First arg checks ensures no overlap between bigraph phrases and trigram phrases.
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=1)

# Create function pointer i.e. phraser that converts document to list of bigram
bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# print(data_bigrams_trigrams[0])

# new_words = []
# for word in data_bigrams_trigrams[0]:
#     if '_' in word:
#         new_words.append(word)
# print(data_bigrams)


In [10]:
# TF-IDF Removal - to remove words that are not important to the topic
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

# Goes through words and looks at words that occur too frequentely and removes them
# Corpus becomes new corpus where those irrelevant words are removed
# low_value = 0.03
low_value = 0.2 # TODO
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow




[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]


In [ ]:
# id2word = corpora.Dictionary(data_words)
# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text) # Create a bag of words all (word: f pairs) for 1 doc
#     corpus.append(new)

# print(corpus[0][0:20])
# #(0,2) - 0 = index of word in the dictionary, 2 = frequency of word in that text

# word = id2word[[0][:1][0]] # i.e. text 1 -> first tuple -> first value in tuple
# print(word) # i.e. able is the first element in teh dictionary 

In [13]:
lda_model = gensim.models.LdaModel(corpus=corpus
                                    ,id2word=id2word
                                    ,num_topics=40
                                    ,random_state=100
                                    ,update_every=1
                                    ,chunksize=100
                                    ,passes=10
                                    ,alpha="auto")

### Idea
I think I can separate the review data set into one star vs 5 star reviews and look at if there are any differences in the topics that are mentioned.

In [ ]:
# Visualising the data
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/root/miniconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/root/miniconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/root/miniconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/root/miniconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/root/miniconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version class

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
41    -0.418713  0.316581       1        1  6.977907
49    -0.172842 -0.489264       2        1  5.596569
15    -0.351345 -0.311485       3        1  5.298526
45     0.521360 -0.000888       4        1  4.380584
28    -0.002937  0.476672       5        1  4.346743
27     0.434811 -0.179236       6        1  4.216887
22     0.303328 -0.427927       7        1  4.164545
23    -0.483052  0.126780       8        1  4.033589
38    -0.424129 -0.198777       9        1  3.630723
9      0.119664  0.411445      10        1  3.591297
31     0.262481  0.408676      11        1  3.571431
26    -0.280474  0.361588      12        1  3.144995
44    -0.162706  0.402786      13        1  3.115734
39     0.122302 -0.264106      14        1  3.065309
21     0.425988  0.188569      15        1  2.978918
37    -0.222563 -0.337886      16        1  2.971039
40     0.006705 -0.447835      17        1  2.709977
7     -0.385063 -0.060721      18        1  2.659205
8     -0.291716 -0.187348      19        1  2.436671
32    -0.381675  0.110555      20        1  2.324405
19     0.203370 -0.171435      21        1  2.260876
1     -0.026552 -0.326864      22        1  2.091334
0      0.289201 -0.079130      23        1  1.967349
4      0.123303 -0.387531      24        1  1.917536
16    -0.284365  0.021186      25        1  1.560511
5     -0.042832  0.299334      26        1  1.532995
18     0.235956  0.098620      27        1  1.504382
2      0.270241 -0.234091      28        1  1.293392
36     0.175433  0.114890      29        1  1.266846
25     0.266999  0.006514      30        1  1.235578
12    -0.097908 -0.223576      31        1  1.067586
17    -0.135319  0.162922      32        1  1.065903
13     0.244882  0.264055      33        1  0.973369
29    -0.124454  0.060200      34        1  0.826288
43    -0.141001 -0.029370      35        1  0.799947
48     0.129048  0.168447      36        1  0.773792
30    -0.004780  0.169835      37        1  0.444931
11     0.001309  0.069316      38        1  0.367155
46     0.064927  0.016214      39        1  0.298685
10     0.021192  0.009312      40        1  0.139681
6      0.021190  0.009310      41        1  0.139681
24     0.021193  0.009286      42        1  0.139681
14     0.021192  0.009288      43        1  0.139681
42     0.021195  0.009298      44        1  0.139681
35     0.021191  0.009303      45        1  0.139681
3      0.021192  0.009288      46        1  0.139681
34     0.021194  0.009291      47        1  0.139681
47     0.021192  0.009291      48        1  0.139681
33     0.021189  0.009310      49        1  0.139681
20     0.021193  0.009311      50        1  0.139681, topic_info=              Term         Freq        Total Category  logprob  loglift
28             app  1569.000000  1569.000000  Default  30.0000  30.0000
66         account  1142.000000  1142.000000  Default  29.0000  29.0000
7           crypto  1213.000000  1213.000000  Default  28.0000  28.0000
122           star   776.000000   776.000000  Default  27.0000  27.0000
12           money   874.000000   874.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
24      debit_card     0.013728   133.327444  Topic50  -8.3547  -2.6076
26         payment     0.013728   185.047193  Topic50  -8.3547  -2.9354
27   account_month     0.013728     7.945961  Topic50  -8.3547   0.2125
28             app     0.013728  1569.648483  Topic50  -8.3547  -5.0734
29         company     0.013728   303.593344  Topic50  -8.3547  -3.4305

[2403 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
208      15  0.981584        ability
5        28  0.972595         access
66       11  0.998565        account
27        6  0.755101  account_month
45       10  0.982007         action
...     ...       ...            ...
225

In [12]:
# Save the model into memory
lda_model.save("models/lda_all_50_topics.model");

### Manual topic allocation


In [18]:
# Spacy load creates an an nlp parser i.e. function pointer 
spacy.load
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

text = "registration register registered verification the card"
doc = nlp(text) # Create document from the raw text/string

allowed_postags = ["NOUN"]
new_text = []
for token in doc:
    if token.pos_ in allowed_postags and token.pos_ not in stopwords:
        new_text.append(token.lemma_) # Appends only lemma to new_text
final = " ".join(new_text)

print(final)

registration verification card


In [ ]:
# Further preprocessing


def pre_process_unseen(unseen_document, useless_words=["app", "shib", "day", "time", "kraken", "hour", "money"]):
    words = " ".join(lemmatization([unseen_document]))
    cleaned_words = [" ".join(gen_words([w])[0])]
    cleaned_words = [word for word in cleaned_words if word not in useless_words]
    return cleaned_words


In [ ]:
labels = {1: "transaction", 2: "trust", 3: "fee", 4: "idk", 5: "idk", 6: "app"}
for i in range(len(lda_model.get_topics())):
    print(lda_model.print_topic(i, 5))

0.226*"system" + 0.183*"place" + 0.096*"solution" + 0.062*"opinion" + 0.043*"business_practice"
0.305*"customer_service" + 0.117*"idea" + 0.113*"usd" + 0.091*"selling" + 0.061*"section"
0.344*"future" + 0.147*"trouble" + 0.138*"need" + 0.117*"access" + 0.029*"pop"
0.000*"model" + 0.000*"aap" + 0.000*"continue" + 0.000*"pleaze" + 0.000*"recording"
0.437*"way" + 0.174*"waste_time" + 0.131*"investment" + 0.094*"moment" + 0.000*"draw"
0.192*"function" + 0.113*"document" + 0.091*"photo" + 0.087*"button" + 0.085*"setting"
0.000*"model" + 0.000*"aap" + 0.000*"continue" + 0.000*"pleaze" + 0.000*"recording"
0.625*"issue" + 0.192*"bug" + 0.011*"fix" + 0.000*"model" + 0.000*"continue"
0.482*"support" + 0.268*"reason" + 0.056*"question" + 0.017*"resolution" + 0.000*"recording"
0.205*"application" + 0.199*"ui" + 0.138*"datum" + 0.119*"other" + 0.078*"action"
0.000*"model" + 0.000*"aap" + 0.000*"continue" + 0.000*"pleaze" + 0.000*"recording"
0.160*"fact" + 0.148*"lot_money" + 0.000*"recording" + 0.0

In [ ]:
unseen_document = "The fee is too high"
bow_vector = id2word.doc2bow(pre_process_unseen(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    # lda_mode[bow_vector] % gets the topic probability for a document
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))
# TODO: Think about how to classify using 

Score: 0.07185057550668716	 Topic: 0.186*"feature" + 0.166*"transaction" + 0.159*"interface" + 0.146*"thing" + 0.129*"review"
Score: 0.057366497814655304	 Topic: 0.518*"crypto" + 0.165*"currency" + 0.121*"trading" + 0.046*"cash" + 0.044*"profit"
Score: 0.05701189115643501	 Topic: 0.321*"website" + 0.278*"update" + 0.173*"phone" + 0.121*"year" + 0.000*"alarm"
Score: 0.04424825310707092	 Topic: 0.481*"money" + 0.336*"problem" + 0.037*"form" + 0.035*"luck" + 0.000*"model"
Score: 0.04346691071987152	 Topic: 0.358*"fee" + 0.290*"platform" + 0.144*"trade" + 0.075*"desktop" + 0.036*"check"
Score: 0.042563263326883316	 Topic: 0.446*"time" + 0.419*"option" + 0.050*"glitch" + 0.000*"aap" + 0.000*"draw"
Score: 0.04083874449133873	 Topic: 0.901*"app" + 0.030*"token" + 0.000*"recording" + 0.000*"continue" + 0.000*"aap"
Score: 0.04082973301410675	 Topic: 0.410*"day" + 0.201*"coinbase" + 0.141*"info" + 0.072*"part" + 0.037*"support_team"
Score: 0.037836700677871704	 Topic: 0.286*"experience" + 0.215*

In [ ]:
# Save the model into memory
lda_model.save(f"models/{csv_file[:-4]}.model");

NameError: name 'csv_file' is not defined